In [1]:
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String

In [2]:
def transform(img, angle):
    
# Once a color image has been read, it passes through the 
# transform function. The transform function has two parts:
# pre-processing also known as pp which filters the image based
# of certain values in the hue, saturation, and value (hsv) 
# colorspace and the dbscan which clusters the processed image
# and classifies the clusters as either a buoy, a green buoy,
# a red buoy, or noise. No changes are needed.

    start_time = time.time()
    bridge = cv_bridge.CvBridge()
    
    width = img.height
    height = img.width
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    
    
    
    
    
    
    overlay = False
    image = pp(img,width,height)
    img2 = dbscan(image, img, overlay,angle)
    return img2
def pp(image, width, height):
    
# The first function in transform is pp or preprocessing. The 
# first line truncates the lower half of the image. The reason
# this is because the plantoons from the wamv are always 
# blocking the camera vision and become noisy data. The
# function changes the image to the HSV colorspace and 
# filters out the image. No changes are needed. It returns 
# the filtered image in the BGR colorspace.The input is the
# color image.

    image = np.array(image[0:width][0:int(height*0.65)])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)

    return(image)
def dbscan(image, original, overlay, angle):
    
# The second function in transform is dbscan. The inputs are 
# the filtered colored image and the original color image
# read from the imread statement. The output is the original
# image.This is the end of this classification algorithm.

###########################################################

# Part 1: Thresholding the image
##########################################################
# The reason we threshold the image before applying DBSCAN to 
# the points is because DBSCAN is a computational demanding 
# function. This part takes the filtered image, makes a 
# copy of it, and converts it into a grayscale image. The gray-
# scale image is thresholded at a value of 10. 0 is black and
# 255 is white. This takes the ~400,000 matrix points and 
# leaves only ~10,000 nonzero matrix points. 

    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

# After thresholding the image, we pull out those nonzero
# points and assign it to the nonzero array variable. This
# array has dimensions of n rows and 2 columns. The first 
# column is the y coordinate and the second column is the x
# coordinate. These two arrays would be combined by 
# np.column_stack which merges the two arrays at their columns.

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))
#########################################################

# Creating the BDSCAN object and Sorting Array with Labels
#########################################################    
# After thresholding the filtered image and creating an
# array of nonzero coordinates, this algorithm applies a
# clustering algorithm that filters the image based off the
# array X. If X is empty, this function ends and output is 
# the original input. The first line creates the DBSCAN
# object with an epsilon of 3 and minimum samples of 20. 
# Eps refers to the size of the neighborhood and the 
# min_samples refers to the minimum number of samples required
# for that neighborhood to be considered a cluster. The user
# is free to change the two values for eps and min_samples. The
# .fit(X) applies the matrix points to the DBSCAN object.
# One of the parameters of the db scan object are the labels.
# This is invoked by the line labels = db.labels. This creates
# an array of labels that assigns a number to the image index.
# n clusters have labels that start at 0 and end with n-1.For 
# example, if an image has four clusters, the labels would
# include 0,1,2, and 3. -1 is dedicated for noise. 

    if (len(X) > 0):
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_

# The array variable combines the x and y coordinates and the 
# labels in a single array aligned by column. The array is
# sorted with the sort variable with the values in the third
# column or the labels. For example, an image with four
# clusters and noise would be labeled in order from (-1,0,1,2,3).
# The while loop iterates from the beginning of the sorted loop
# and deletes every instance of -1 or until the list is empty.
# This while loop is needed because the noise data is
# irrelevant to the cluster algorithm and the user does not 
# know how much noise there is in the image. The image could 
# have no noise or it could all be noise. The output of this
# section is an array with coordinates sorted together in 
# clusters.

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]
#########################################################

# Iterating through and classifying each cluster
#########################################################
# After iterating through sort, the program is either left with
# an empty array or an array of coordinates sorted and grouped
# by each cluster. The user needs the number of clusters. The 
# next three lines in the big if block does this. unique_labels
# sorts the set of labels and removes -1 if present. The next 
# two lines creates a copy of the sort array and removes the 
# labels column from the array.

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

# The program assigns three variables. The first one i iterates
# through sort and compare its label value to x for each 
# iteration until the very end of the sort array. The second
# x refers to the cluster that the program is currently class-
# ifying. It starts with 0 and continues classifying until it 
# has classified all clusters. The third variable is l and this
# array becomes all the points within a cluster. Every time the 
# variable i iterates through sort and is equal to the value of
# the current cluster x, the coordinates are appended to the 
# array l. Since this array is sorted, the first time the x 
# variable equals a different value- the cluster has been 
# accounted for. This array of l contains all the points within
# the cluster. This array and the filtered image gets passed
# through the color function. The result of the color function
# gets passed through the rectangle function with the array,
# filtered image, and original image. The x variable is 
# incremented for the next cluster and the l array is set equal 
# to zero. Once the while loop has iterated through all 
# clusters, the result of the rectangle function is set to the 
# original image and the original image is returned to the 
# transform function which is returned to the read function 
# which writes the image.

            i = 0
            x = 0
            l = []
            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    clr = color_hsv(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image,original, overlay,angle)
                    l = []
            original = tb
    return original
def color(mat,image):
    
    ############################################################

# For the color function, the points of the cluster and the 
# filtered image are passed through. In this function, the 
# average red, green, and blue pixel values are calculated
# from the image inside of the cluster. If the average value
# fits within a certain range, the string 'red' or 'green' is 
# returned. Otherwise, the function returns the string 'none'
# The user is free to change the range of the BGR values that
# determine whether the buoy is red or green.

    sumred = 0
    sumgrn = 0
    sumblu = 0
    for x in range(len(mat)):
        sumred += image [mat[x][1]] [mat[x][0]] [2]
        sumgrn += image [mat[x][1]] [mat[x][0]] [1]
        sumblu += image [mat[x][1]] [mat[x][0]] [0]
    if ((sumred/len(mat)) > 80 and (sumred/len(mat)) < 180 and (sumgrn/len(mat)) > 30 and (sumgrn/len(mat)) < 115  and (sumblu/len(mat)) > 30 and (sumblu/len(mat)) < 110):
        return 'red'
    if ((sumred/len(mat)) > 30 and (sumred/len(mat)) < 80 and (sumgrn/len(mat)) > 60 and (sumgrn/len(mat)) < 180  and (sumblu/len(mat)) > 50 and (sumblu/len(mat)) < 130):
        return 'green'
    else:
        return 'none'
def color_hsv(mat, image):
    
# The rectangle function's parameters are the array of coordinates
# of the cluster, the color of the cluster, the filtered image,
# and the original image. The boundingRect function applies
# a non-rotated rectangle to the image points and returns four
# values in an array, the x and y coordinates of the leftmost
# corner and the width and height. These values are converted
# to ints. The font is set to Hershey Simple for the text and
# the tag is set to zero. The tag is useful in the classifica-
# tion of the buoy. The first classification of the buoy is 
# the proportion of the width and height. If the height is
# between 1.5 times the width and 3.5 times the width, the
# cluster is considered a buoy. If the cluster is given a 
# 'red' or 'green' color, it would be classified as a 'red buoy'
# or 'green buoy' respectively. Otherwise, it would simply be
# classified as a buoy. If any of these three options occur, 
# the tag is set to 1. If the height is not in that range
# mentioned above and the color is 'none', the cluster is
# classified as noise. The last function rectangle makes a 
# rectangle over the original image. The original image is sent
# back to the DBSCAN function.


    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    sumhue = 0
    sumsat = 0
    sumval = 0
    mat_add = 0
    for x in range(len(mat)):
        if(image [mat[x][1]] [mat[x][0]] [2] > 5):
            mat_add = mat_add+1
            sumval += image [mat[x][1]] [mat[x][0]] [2]
            sumsat += image [mat[x][1]] [mat[x][0]] [1]
            sumhue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sumsat/mat_add
        val = sumval/mat_add
        hue = sumhue/mat_add
        #return (hue, sat, val)
        if (sat > 5 and val > 5 and (hue < 60 or hue >= 170)):
            return 'red'
        if (sat > 5 and val > 5 and (70 < hue < 90)):
            return 'green'
        else:
            return (hue, sat, val)

    return None

In [3]:
# Initialization Stuff
rospy.init_node('liveproc')
listener = tf.TransformListener()
listener.waitForTransform("/odom", "/front_left_camera_link", rospy.Time(), rospy.Duration(2.0))

occ_map = None
camera_range = 75
    
np.set_printoptions(threshold=sys.maxsize)

In [4]:
def rectangle(mat,color,image,original, overlay, angle):
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)
#     x_bearing = min_rect[0][1]
    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 1
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
            elif(color =='red'):
                tag = 1
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
#             cv.putText(original,str(x_bearing) + " degrees",(np.int0(x-width/2),np.int0(y-height*math.log(height,10))), font, 0.5,(19,69,139),2,cv.LINE_AA)
            cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (x+width/2-640.0)/1280.0*80.0
            angle.append(bearing)
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
        return original, angle
    else:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 1
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            elif(color =='red'):
                tag = 1
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
#             cv.putText(image,str(x_bearing) + " degrees",(np.int0(x-width/2),np.int0(y-height*math.log(height,10))), font, 0.5,(255,255,255),2,cv.LINE_AA)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (x+width/2-640.0)/1280.0*80.0
            angle.append(bearing)
        return image, angle

In [5]:
def map_filter(p_map, trans, robot_yaw, yaw_offset):

    map_array = p_map.data
    origin = p_map.info.origin.position

    map_width = p_map.info.width
    map_height = p_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))
    

    print_to_file = False
    robot_pos_x = trans[0]-origin.x
    robot_pos_y = trans[1]-origin.y
    if (robot_yaw < 0):
        robot_yaw = robot_yaw + 360
    i = 0
    f = None
    slices = []
    while (i < len(yaw_offset)):
        buoy_yaw = robot_yaw - yaw_offset[i]
        if ((buoy_yaw) < 0):
            buoy_yaw = buoy_yaw + 360
        global camera_range
        point2_x = camera_range * math.sin(math.radians(buoy_yaw+45))
        point2_y = camera_range * math.cos(math.radians(buoy_yaw+45))
        point1 = np.array([round(robot_pos_x), round(robot_pos_y)])
        point2 = np.array([round(robot_pos_x+point2_x), round(robot_pos_y+point2_y)])
        sliced = createLineIterator(point1.astype(int),point2.astype(int),map_np)
        slices.append(sliced)
        i+=1

    return slices

In [6]:
def createLineIterator(P1, P2, img):
# """
# Produces and array that consists of the coordinates and intensities of
# each pixel in a line between two points

# Parameters:
#     -P1: a numpy array that consists of the coordinate of the first point (x,y)
#     -P2: a numpy array that consists of the coordinate of the second point (x,y)
#     -img: the image being processed

# Returns:
#     -it: a numpy array that consists of the coordinates and intensities of each
#     pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
# """
   #define local variables for readability
    #print img.shape
    image_height = img.shape[0]
    image_width = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]
    
   #difference and absolute difference between points
   #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3), dtype=np.int32)
    itbuffer.fill(np.nan)

   #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

   #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<image_width) & (colY<image_height)]

   #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]
    return itbuffer

In [7]:
def get_map(p_map, origin, robot_x, robot_y):
    global camera_range
    map_array = p_map.data
    map_width = p_map.info.width
    map_height = p_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))

    map_img = np.zeros((map_height, map_width, 3))
#     map_img[map_np == -1] = [255,128,0]
#     map_img[map_np == 0] = [204,0,0]
#     map_img[map_np == 100] = [0,0,0]
    
    map_img[map_np == -1] = [255,255,255]
    map_img[map_np == 0] = [255,255,255]
    map_img[map_np == 100] = [0,0,0]
    
    map_img[int(-origin.y)][int(-origin.x)] = [0,0,0]
    
    map_img[robot_y][robot_x] = [0,0,0]
    map_
#     map_img = cv.getRotationMatrix2D((map_height/2,map_width/2),90,1)
    
#     slices_size = len(slices)
# #     print(slices_size)
#     slices = np.reshape(np.delete(slices, 0, 0), ((len(slices)-1)/3,3))
# #     print slices
# #     time.sleep(100000)
    
# #     for i in range(int(slices_size)):
# #         try:
# #             found = False
# #             arr = slices[i]
# #             for slice in arr:
# #                 print slice
# #                 if not found and (slice[2] == -1 or slice[2] == 0):
# #                     #map_img[slice[1]][slice[2]]=[102,0,102]
# #                     map_img[slice[1]][slice[0]] = [255,255,255]
# #                 elif slice[2] == 100:
# #                     map_img[slice[1]][slice[0]]=[0,0,255]
# #                     found = True
# #         except Exception as ex:
# #             print ex
# #             print slices[i]
# #             print
    
    return(map_img)

In [8]:
def callback(rect):
    global occ_map
    #print "right before"
    start_time = time.time()
    display_rect = False
    angle = []
    origin = occ_map.info.origin.position
    rect, bearings = transform(rect,angle)
    
    
    global listener 
    if not rospy.is_shutdown():       
        try:
            now = rospy.Time.now()
            listener.waitForTransform("/odom", "/base_link", now, rospy.Duration(2.0))
            (trans,rot) = listener.lookupTransform("/odom", "/base_link", now)
            euler = tf.transformations.euler_from_quaternion(rot)
            print trans
            robot_pos_x = trans[0]-origin.x
            robot_pos_y = trans[1]-origin.y
            map_image = get_map(occ_map, origin, int(robot_pos_x), int(robot_pos_y))
            cv.namedWindow("map",cv.WINDOW_NORMAL)
            cv.imshow("map", map_image)
            cv.waitKey(1)
        except tf.LookupException as l:
            print l
        except tf.ConnectivityException as c:
            print c
        except tf.ExtrapolationException as e:
            print e
        except Exception as ex:
            print ex

In [9]:
def map_callback(p_map):
    global occ_map
    occ_map = p_map

In [ ]:
if __name__ == '__main__':
    proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin()
    

[146.15627355943914, 103.53814187435347, -0.07224746794569298]
[146.16140153605323, 103.54050549783324, -0.08848915931531644]
[146.1644237098048, 103.54215585139067, -0.0998576456009965]
[146.16766354988974, 103.54433634701861, -0.11293695827987517]
[146.16963395998164, 103.54614515797655, -0.12671829169150167]
[146.16964555400114, 103.54669775617337, -0.14396162813246838]
[146.16774371343084, 103.54710689545166, -0.15465482067041944]
[146.16201298498532, 103.54639881558184, -0.16552327352549936]
[146.15456340627824, 103.5445456726232, -0.1708391643779828]
[146.1503897457839, 103.54332091278165, -0.17220763681128767]
[146.1410909653615, 103.54047759902409, -0.16636170333731937]
[146.1288198941051, 103.53634258505198, -0.15128009579377738]
[146.12225405407872, 103.53393484894008, -0.1342129224000444]
[146.11350083595946, 103.530517097544, -0.10989126607024248]
[146.10940361987494, 103.52865171465166, -0.08588340883503712]
[146.10779889297467, 103.52757340094018, -0.05899194629312188]
[1

[145.96962665400147, 103.47731695828352, -0.08091201734542777]
[145.9664587185021, 103.47581650357522, -0.06098579416562576]
[145.96729673129119, 103.47539479183664, -0.04437869272929018]
[145.96672435004785, 103.47485379603962, -0.021940523712049277]
[145.97130354799324, 103.4756926411275, -0.011542753295670033]
[145.9802332774488, 103.47778055512819, -0.004245277018308778]
[145.9855483265835, 103.47903831002385, -0.0016644271077742175]
[145.99196881667058, 103.48065504127709, -0.00637942868676101]
[146.00281691679203, 103.48340815449885, -0.02078958817033531]
[146.00840131444454, 103.48464153198235, -0.03901115119369986]
[146.01261508211545, 103.48543209296287, -0.056806821129183074]
[146.01413735018454, 103.48547201689655, -0.081777578681021]
[146.01434261949493, 103.48476376727821, -0.10144098450619807]
[146.01141320342924, 103.48330270353088, -0.1275169769726946]
[146.00527810580576, 103.48066340668697, -0.152488734384983]
[145.99696280734238, 103.47762149777719, -0.16405881620897

[145.80541177457852, 103.4123234025146, -0.11523333675459432]
[145.80577115935716, 103.41364637991114, -0.10609999466046165]
[145.80751284334093, 103.41555736638882, -0.09854974094701197]
[145.80928241900602, 103.41702739654433, -0.08985705476730922]
[145.81309032729183, 103.41939343827417, -0.08660374344116963]
[145.81687022820986, 103.42150774962552, -0.08372396705340591]
[145.817673470442, 103.42213922393495, -0.08210112577273505]
[145.8204271632583, 103.42321870277542, -0.08381147815713166]
[145.822018219362, 103.42345733640148, -0.08681394493150234]
[145.82086627004082, 103.42252812898074, -0.08969795525009182]
[145.8182624015302, 103.42055521086002, -0.09228131819188076]
[145.8132844239941, 103.41702471083576, -0.0920043481719654]
[145.80334762517828, 103.4103810951305, -0.08677577606279793]
[145.7978269404176, 103.40647649527983, -0.0807693956982815]
[145.79540619151186, 103.4040766198143, -0.07684115097340305]
[145.79545710538315, 103.40367256725669, -0.07582301966929567]
[145.

[145.64784985158212, 103.35659341936446, -0.18332908872664083]
[145.63561742410914, 103.35286768255708, -0.16738375468789174]
[145.62897834220416, 103.35053342379736, -0.14991455302056778]
[145.62068499375846, 103.34758213278103, -0.12316533683993422]
[145.61416099773172, 103.34508110781582, -0.09321895374316766]
[145.6130247330874, 103.34438615924842, -0.07293879781175971]
[145.61356679597301, 103.34416096212742, -0.05298055428922442]
[145.61683609181426, 103.3448039805689, -0.032541918474847935]
[145.6228268003177, 103.34632725122427, -0.019043561957707185]
[145.62504439293699, 103.34683682684172, -0.00913031156977545]
[145.63335002802384, 103.34911181611965, -0.011421158672790494]
[145.6451324598965, 103.35225879790728, -0.022942442271833126]
[145.64988903680296, 103.35337264965887, -0.03655122246809737]
[145.65516794593114, 103.35448379532433, -0.05351380397117299]
[145.65888920525998, 103.35475227597107, -0.07760123295973753]
[145.65766107462625, 103.3537745366446, -0.100238353816

[145.4620643535292, 103.28242593457593, -0.1571949930978801]
[145.45627726811892, 103.2814990202302, -0.1431295274277612]
[145.4533586617631, 103.28137791711644, -0.12741421401849196]
[145.45185938843548, 103.2822209239439, -0.11044401146364144]
[145.45310264641668, 103.2837884214284, -0.10022799344678356]
[145.4545908195465, 103.28540958677414, -0.09177512828596893]
[145.4584584167387, 103.28772511292418, -0.08545891016313605]
[145.45941835243698, 103.28842435992347, -0.07913715988898355]
[145.46312249391156, 103.29042725792182, -0.0745066846320411]
[145.4660664282183, 103.2917289189644, -0.07281572991823633]
[145.4682707044776, 103.29237586580383, -0.07324086629338097]
[145.46817639941253, 103.2919285245779, -0.0734554999591832]
[145.467664145836, 103.29148700968311, -0.07485638867584113]
[145.46588012285085, 103.28972074061338, -0.07504231313089321]
[145.4616114088546, 103.28626709184957, -0.07426090036705603]
[145.45769601245667, 103.28355079851023, -0.07332838065823398]
[145.45300

...

[145.15366874596512, 103.17281425544355, -0.1705478795807957]
[145.1459273536713, 103.17022192161664, -0.1586690139096421]
[145.1362088295092, 103.16681162943676, -0.1383822614478786]
[145.1264914059385, 103.16314190373949, -0.11228236552982182]
[145.1220801129475, 103.16121345406165, -0.08496492668861472]
[145.11925345422333, 103.15968245494237, -0.05782792699007442]
[145.11958701024514, 103.15916186894042, -0.03376086779358188]
[145.12283714306275, 103.15961423502006, -0.01565841762555762]
[145.12842666138366, 103.16085010037241, -0.004594608347489465]
[145.13623528145695, 103.16276433977914, -0.0013538723896029514]
[145.14219033887886, 103.16427395307643, -0.003908066868682104]
[145.14741655547482, 103.16559190712375, -0.009825155617515407]
[145.1537209668162, 103.16712398593714, -0.021183647461193816]
[145.1604137129217, 103.1685931695178, -0.041041228561778935]
[145.16598100760663, 103.16946313349436, -0.06584433772446573]
[145.16596704641103, 103.16880940120359, -0.09259818378401

[144.9620529765614, 103.09594959150297, -0.130427705053317]
[144.95977017170955, 103.09662969210548, -0.12081341310270424]
[144.95909065242645, 103.09727621438158, -0.10963778728306608]
[144.95976591363026, 103.09846659222397, -0.10278073653313143]
[144.9596352431612, 103.09903677093868, -0.09534581125538404]
[144.96443926418743, 103.10217348119923, -0.09247539290289233]
[144.96680768989194, 103.1037366661303, -0.08905803679819889]
[144.9689731637664, 103.1050667493345, -0.08604440864941994]
[144.9722123955247, 103.10672831692548, -0.08675231461408121]
[144.97398993312584, 103.10747856486249, -0.08799921013121845]
[144.9741662994961, 103.10745511847628, -0.09060096400574653]
[144.97307591427244, 103.10635397775553, -0.09137510823758253]
[144.97081100515913, 103.10485681998249, -0.09293483774149322]
[144.9676665423611, 103.10272443549616, -0.09302058725598862]
[144.9621758967055, 103.09902324140725, -0.0912603630087234]
[144.95746761472319, 103.09584852314805, -0.08752343875560152]
[144

[144.80958350550293, 103.03436017674528, -0.0589829064848444]
[144.81287823583963, 103.03521349814278, -0.06542276521089437]
[144.8143266985482, 103.03563257572046, -0.07115255011782287]
[144.81755471041012, 103.03694824720432, -0.08000181030574469]
[144.82239976983047, 103.0393121484293, -0.09186658330958222]
[144.82426449406637, 103.04023335201975, -0.10301417789159464]
[144.82741755463482, 103.04220741679711, -0.1159371371290906]
[144.83041288851626, 103.0442359566405, -0.13085839682737477]
[144.83122051715267, 103.04539963731918, -0.14335828828098712]
[144.83048521917632, 103.04602746856696, -0.15968334019905414]
[144.82903282278812, 103.04644576676333, -0.17053637297700275]
[144.8276191119474, 103.04652250122774, -0.18147643493472754]
[144.82250721123103, 103.04590250557033, -0.18785981367849836]
[144.81717014316843, 103.04463994523405, -0.1912809843982627]
[144.81332118634518, 103.04379544302556, -0.19489165155443827]
[144.80670247962462, 103.04204133109774, -0.19197576521380905]

[144.68867877794582, 102.99336626425148, -0.12651124862765156]
[144.6886281205167, 102.99385179218892, -0.14573325300785986]
[144.68629249197528, 102.9940195101005, -0.15562126236860713]
[144.68211961936535, 102.9935047317091, -0.16403060394576274]
[144.67705315039464, 102.99243922982488, -0.1742773986392585]
[144.6688338808261, 102.99010161328027, -0.1739285937916669]
[144.66385311923506, 102.98867372406792, -0.17479264179573037]
[144.6544405427514, 102.98563368825418, -0.16523603665631986]
[144.6420026801042, 102.9814470590755, -0.1475330095993334]
[144.63568997468633, 102.97916014376844, -0.12821020789566362]
[144.62898884070694, 102.97661724514599, -0.1034204510743046]
[144.6252925078702, 102.97500606046609, -0.07918246656786594]
[144.62387522572533, 102.9741647303836, -0.0567719990865872]
[144.6258625638245, 102.97452743966524, -0.04489259098203151]
[144.62653995610705, 102.97459055568387, -0.03389188095955563]
[144.6308165295685, 102.9756961098169, -0.026478524056766522]
[144.636

[144.4880469762906, 102.92285153259098, -0.01336955301052485]
[144.4942224028022, 102.9242944418898, -0.0033577493580924864]
[144.50246915455205, 102.92643869849607, -0.003066455535975943]
[144.51106379149434, 102.92861199693075, -0.007885756169553252]
[144.51934880463352, 102.93061316949851, -0.020417605817950615]
[144.5256534719129, 102.93191093990583, -0.03923344148372446]
[144.52926786120824, 102.93221533269367, -0.06708617922335873]
[144.53035481718442, 102.93209945428865, -0.08690661991247368]
[144.52890339740776, 102.9306726050893, -0.10504480382111375]
[144.52508198876185, 102.92882643063403, -0.12925624612564549]
[144.51802423729603, 102.92578725366405, -0.15068820196389945]
[144.5065448768437, 102.92144334624813, -0.16297843049845379]
[144.49983633839653, 102.91923127424315, -0.16884568893912966]
[144.49198836108383, 102.9167225617577, -0.17066319771995192]
[144.48079060057518, 102.91358159678094, -0.16660583617793304]
[144.47479052960693, 102.91222144706612, -0.1598880257575